<a href="https://colab.research.google.com/github/bhartiansh/cnn_pruning_cifar10/blob/main/pruning3(MAG_50).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!git clone https://github.com/bhartiansh/cnn_pruning_cifar10.git
%cd cnn_pruning_cifar10

Cloning into 'cnn_pruning_cifar10'...
remote: Enumerating objects: 105, done.
remote: Counting objects: 100% (105/105), done.
remote: Compressing objects: 100% (101/101), done.
remote: Total 105 (delta 42), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (105/105), 64.10 KiB | 5.34 MiB/s, done.
Resolving deltas: 100% (42/42), done.
/content/cnn_pruning_cifar10


In [3]:
!pip install -q tensorflow-model-optimization

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.5/242.5 kB 5.2 MB/s eta 0:00:00


In [4]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import tensorflow_model_optimization as tfmot
from models.resnet56_baseline import build_resnet56

In [5]:
def get_global_magnitude_scores(model):
    scores = []
    for weight in model.trainable_weights:
        if 'kernel' in weight.name:  # Ignore biases and BN params
            scores.append(tf.reshape(tf.abs(weight), [-1]))
    return tf.concat(scores, axis=0)


In [6]:
def apply_global_magnitude_pruning(model, sparsity):
    all_scores = get_global_magnitude_scores(model)
    k = int((1 - sparsity) * all_scores.shape[0])
    threshold = tf.sort(all_scores, direction='ASCENDING')[k]

    # Apply mask
    for weight in model.trainable_weights:
        if 'kernel' in weight.name:
            mask = tf.cast(tf.abs(weight) >= threshold, tf.float32)
            pruned = weight * mask
            weight.assign(pruned)


In [7]:
def train_magnitude_pruned_model(build_model_fn, x_train, y_train, x_val, y_val,
                                 sparsity=0.5, epochs=50, batch_size=128, save_path='magprune_model.h5'):
    model = build_model_fn()  # build fresh model
    apply_global_magnitude_pruning(model, sparsity)

    early_stop = tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)
    checkpoint = tf.keras.callbacks.ModelCheckpoint(save_path, save_best_only=True)

    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    history = model.fit(x_train, y_train,
                        validation_data=(x_val, y_val),
                        epochs=epochs,
                        batch_size=batch_size,
                        callbacks=[early_stop, checkpoint],
                        verbose=2)

    return model, history


In [8]:
!ls


 data			  'pruning4(L1_Norm_Filter).ipynb'
 models			  'pruning5(Random_Pruning_Unstructured).ipynb'
'pruning1(lth).ipynb'	   README.md
'pruning2(SNIP).ipynb'	   ResNet56_baseline_model.ipynb
'pruning3(MAG_50).ipynb'   traning


In [9]:
# Load and normalize CIFAR-10
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Train magnitude-pruned model
model, history = train_magnitude_pruned_model(
    build_model_fn=build_resnet56,  # your model builder
    x_train=x_train,
    y_train=y_train,
    x_val=x_test,
    y_val=y_test,
    sparsity=0.5,  # 50% pruning
    save_path='magprune_model.h5'
)


170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step
Epoch 1/50


391/391 - 105s - 268ms/step - accuracy: 0.4662 - loss: 1.5083 - val_accuracy: 0.4949 - val_loss: 1.4656
Epoch 2/50


391/391 - 47s - 121ms/step - accuracy: 0.6286 - loss: 1.0438 - val_accuracy: 0.5932 - val_loss: 1.1745
Epoch 3/50
391/391 - 40s - 101ms/step - accuracy: 0.7043 - loss: 0.8364 - val_accuracy: 0.4545 - val_loss: 2.3068
Epoch 4/50


391/391 - 29s - 75ms/step - accuracy: 0.7572 - loss: 0.6892 - val_accuracy: 0.6444 - val_loss: 1.0987
Epoch 5/50
391/391 - 41s - 104ms/step - accuracy: 0.7932 - loss: 0.5866 - val_accuracy: 0.6598 - val_loss: 1.1124
Epoch 6/50
391/391 - 41s - 105ms/step - accuracy: 0.8242 - loss: 0.4978 - val_accuracy: 0.6331 - val_loss: 1.3456
Epoch 7/50
391/391 - 42s - 107ms/step - accuracy: 0.8522 - loss: 0.4228 - val_accuracy: 0.6345 - val_loss: 1.2956
Epoch 8/50


391/391 - 42s - 108ms/step - accuracy: 0.8717 - loss: 0.3627 - val_accuracy: 0.6832 - val_loss: 1.0702
Epoch 9/50
391/391 - 29s - 74ms/step - accuracy: 0.8944 - loss: 0.3034 - val_accuracy: 0.6812 - val_loss: 1.3440
Epoch 10/50
391/391 - 41s - 105ms/step - accuracy: 0.9088 - loss: 0.2537 - val_accuracy: 0.6477 - val_loss: 1.3783
Epoch 11/50
391/391 - 31s - 79ms/step - accuracy: 0.9268 - loss: 0.2102 - val_accuracy: 0.6482 - val_loss: 1.5071
Epoch 12/50
391/391 - 39s - 100ms/step - accuracy: 0.9369 - loss: 0.1770 - val_accuracy: 0.6578 - val_loss: 1.6090
Epoch 13/50
391/391 - 42s - 106ms/step - accuracy: 0.9454 - loss: 0.1536 - val_accuracy: 0.7319 - val_loss: 1.1460


In [10]:
# Load and normalize CIFAR-10
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Train magnitude-pruned model
model, history = train_magnitude_pruned_model(
    build_model_fn=build_resnet56,  # your model builder
    x_train=x_train,
    y_train=y_train,
    x_val=x_test,
    y_val=y_test,
    sparsity=0.3,  # 30% pruning
    save_path='magprune_model.h5'
)


Epoch 1/50


391/391 - 96s - 246ms/step - accuracy: 0.4423 - loss: 1.6145 - val_accuracy: 0.4904 - val_loss: 1.4998
Epoch 2/50
391/391 - 31s - 78ms/step - accuracy: 0.6254 - loss: 1.0493 - val_accuracy: 0.4859 - val_loss: 1.6776
Epoch 3/50


391/391 - 40s - 102ms/step - accuracy: 0.7106 - loss: 0.8245 - val_accuracy: 0.6571 - val_loss: 1.0465
Epoch 4/50


391/391 - 31s - 79ms/step - accuracy: 0.7622 - loss: 0.6790 - val_accuracy: 0.6876 - val_loss: 0.9395
Epoch 5/50
391/391 - 39s - 100ms/step - accuracy: 0.7947 - loss: 0.5830 - val_accuracy: 0.6481 - val_loss: 1.0437
Epoch 6/50


391/391 - 42s - 108ms/step - accuracy: 0.8237 - loss: 0.4983 - val_accuracy: 0.7342 - val_loss: 0.7998
Epoch 7/50
391/391 - 41s - 105ms/step - accuracy: 0.8472 - loss: 0.4334 - val_accuracy: 0.7278 - val_loss: 0.8149
Epoch 8/50
391/391 - 40s - 102ms/step - accuracy: 0.8668 - loss: 0.3749 - val_accuracy: 0.7053 - val_loss: 1.0699
Epoch 9/50
391/391 - 41s - 105ms/step - accuracy: 0.8864 - loss: 0.3198 - val_accuracy: 0.5918 - val_loss: 2.0061
Epoch 10/50
391/391 - 41s - 104ms/step - accuracy: 0.9046 - loss: 0.2696 - val_accuracy: 0.7124 - val_loss: 1.1424
Epoch 11/50
391/391 - 42s - 108ms/step - accuracy: 0.9191 - loss: 0.2276 - val_accuracy: 0.7535 - val_loss: 0.9529


In [11]:
# Load and normalize CIFAR-10
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Train magnitude-pruned model
model, history = train_magnitude_pruned_model(
    build_model_fn=build_resnet56,  # your model builder
    x_train=x_train,
    y_train=y_train,
    x_val=x_test,
    y_val=y_test,
    sparsity=0.7,  # 70% pruning
    save_path='magprune_model.h5'
)


Epoch 1/50


391/391 - 97s - 249ms/step - accuracy: 0.4614 - loss: 1.5363 - val_accuracy: 0.3531 - val_loss: 2.1688
Epoch 2/50


391/391 - 30s - 77ms/step - accuracy: 0.6433 - loss: 1.0056 - val_accuracy: 0.6092 - val_loss: 1.1594
Epoch 3/50
391/391 - 40s - 102ms/step - accuracy: 0.7208 - loss: 0.7935 - val_accuracy: 0.5820 - val_loss: 1.2179
Epoch 4/50


391/391 - 42s - 107ms/step - accuracy: 0.7655 - loss: 0.6681 - val_accuracy: 0.7045 - val_loss: 0.8952
Epoch 5/50
391/391 - 41s - 104ms/step - accuracy: 0.7980 - loss: 0.5739 - val_accuracy: 0.6924 - val_loss: 0.9665
Epoch 6/50
391/391 - 41s - 105ms/step - accuracy: 0.8296 - loss: 0.4876 - val_accuracy: 0.6686 - val_loss: 1.1922
Epoch 7/50
391/391 - 42s - 107ms/step - accuracy: 0.8532 - loss: 0.4189 - val_accuracy: 0.7226 - val_loss: 0.8991
Epoch 8/50
391/391 - 41s - 106ms/step - accuracy: 0.8760 - loss: 0.3508 - val_accuracy: 0.7208 - val_loss: 0.9509
Epoch 9/50
391/391 - 40s - 102ms/step - accuracy: 0.8934 - loss: 0.2967 - val_accuracy: 0.6968 - val_loss: 1.1326


In [ ]:
# 1. Force compatible versions (reset everything to Colab defaults)
!pip install -U --force-reinstall numpy==1.23.5
!pip install -U --force-reinstall tensorflow==2.14.0
!pip install -U tensorflow-model-optimization

# 2. Restart runtime automatically after install
import os
os.kill(os.getpid(), 9)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 28.2 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pymc 5.21.2 requires numpy>=1.25.0, but you have numpy 1.23.5 which is incompatible.
chex 0.1.89 requires numpy>=1.24.1, but you have numpy 1.23.5 which is incompatible.
treescope 0.1.9 requires numpy>=1.25.2, but you have numpy 1.23.5 which is incompatible.
bigframes 1.42.0 requires numpy>=1.24.0, but you have numpy 1.23.5 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 1.23.5 which is incompatible.
imbalanced-learn 0.13.0 requires numpy<3,>=1.24.3, but you have numpy 1.23.5 which is incompatible.
albucore 0.0.23 requires numpy>=1.24.4, but you 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 489.9/489.9 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 54.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 105.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 97.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 72.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 76.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 89.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 

  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.5/126.5 kB 9.1 MB/s eta 0:00:00
Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.3 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.4
    Uninstalling numpy-2.2.4:
      Successfully uninstalled numpy-2.2.4
  Attempting uninstall: absl-py
    Found existing installation: absl-py 2.2.2
    Uninstalling absl-py-2.2.2:
      Successfully uninstalled absl-py-2.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-text 2.18.1 requires tensorflow<2.19,>=2.18.0, but you have tensorflow 2.14.0 which is incompatible.
tf-keras 2.18.0 requires tensorflow<2.19,>=2.18, but you have tensorflow 2.14.0 which is incompatible.
tensorstore 0.1.73 requires